In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("talk")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

from ese.experiment.analysis.analyze_inf import load_cal_inference_stats
# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml results_cfg 

log:
    root: /storage/vbutoi/scratch/ESE/inference
    inference_groups: 
        - '06_12_24_WMH_CalibratedMultiAnno'

options:
    add_dice_loss_rows: True
    drop_nan_metric_rows: True 
    remove_shared_columns: False
    equal_rows_per_cfg_assert: False 

In [ ]:
inference_df = load_cal_inference_stats(
    results_cfg=results_cfg,
    load_cached=True,
)

In [ ]:
# For the purpose of this experiment, we only care about a few columns in particular:
exp_columns = [
    "calibrator",
    "data_id",
    "gt_volume",
    "hard_volume",
    "soft_volume",
    "pretrained_seed", 
    "slice_idx",
    "split",
    "task"
]
# Take these columns of the inference_df, drop other columns.
experiment_df = inference_df[exp_columns]
# Remove the duplicate rows.
experiment_df = experiment_df.drop_duplicates()

In [ ]:
# Accumulate the volumes.
volume_df = experiment_df.groupby(["data_id", "calibrator", "task", "pretrained_seed", "split"]).agg(
    gt_volume=("gt_volume", "sum"),
    hard_volume=("hard_volume", "sum"),
    soft_volume=("soft_volume", "sum"),
).reset_index()

In [ ]:
# Compute a few metrics that are important for comparison.

# Make two new columns, one for the soft volume error and one for the hard volume error.
volume_df['soft_volume_error'] = volume_df['soft_volume'] - volume_df['gt_volume']
volume_df['hard_volume_error'] = volume_df['hard_volume'] - volume_df['gt_volume']

# Make the normalized metric that divides the error by the ground truth volume.
volume_df['soft_norm_volume_error'] = volume_df['soft_volume_error'] / volume_df['gt_volume']
volume_df['hard_norm_volume_error'] = volume_df['hard_volume_error'] / volume_df['gt_volume']

# Library Funcs

In [ ]:
def prepare_error_df(raw_df, groupby_keys):
    # Make a clone of the volume df.
    input_df = raw_df.copy()
    # Melt the dataframe to have a single column for the error.
    error_df = pd.melt(
        input_df,
        id_vars=groupby_keys,
        value_vars=["soft_volume_error", "hard_volume_error"],
        var_name="volume_type",
        value_name="error",
    )
    # Make some columns that are useful for plotting.
    error_df['abs_error'] = error_df['error'].abs()
    error_df['log_abs_error'] = error_df['error'].abs().apply(lambda x: np.log(x + 1))
    # Return the melted dataframe.
    return error_df

In [ ]:
def process_volume_types(input_df):
    # Make a clone of the input_df
    df = input_df.copy()
    # Drop all the rows where calibrator != Uncalibrated AND the volume_type is hard_volume_error.
    df = df[~((df['calibrator'] != 'Uncalibrated') & (df['volume_type'] == 'hard_volume_error'))]
    # Then we augment the volume_type with the calibrator name.
    def volume_type(calibrator, volume_type):
        if calibrator == "Uncalibrated":
            return "Uncalibrated_" + "_".join(volume_type.split("_")[:-1])
        else:
            return calibrator + "_soft_volume"
    df['volume_type'] = df.apply(lambda x: volume_type(x['calibrator'], x['volume_type']), axis=1)
    # Return the augmented dataframe.
    return df

# Experiment 1: Looking at one annotator on Amsterdam, let's look at how the volumetric comparison looks like.

In [ ]:
# Make some columns that are useful for plotting.
raw_melted_error_df = prepare_error_df(volume_df, groupby_keys=[
    "calibrator", 
    "data_id", 
    "pretrained_seed", 
    "gt_volume", 
    "soft_volume", 
    "hard_volume",
    "split",
    "task", 
])
# Process the volume types.
melted_error_df = process_volume_types(raw_melted_error_df)

### First, we look at how we perform with standard difference on the Amsterdam hospital.

In [ ]:
exp_1_df = melted_error_df.select(task='Amsterdam')

In [ ]:
g = sns.catplot(
    exp_1_df,
    x="data_id",
    y="error",
    hue="volume_type",
    col="split",
    aspect=1.5,
    height=8,
    sharex=False,
    sharey=False
)
# For each subplot make a line at y = 0 to show the error.
for ax in g.axes.flat:
    ax.axhline(0, ls='--', color='red')

# Adjust the layout
plt.subplots_adjust(top=0.85)
g.fig.suptitle('Soft/Hard Volumetric Error', fontsize=30)

# Show the plot
plt.show()

In [ ]:
g = sns.catplot(
    exp_1_df,
    x="data_id",
    y="log_abs_error",
    hue="volume_type",
    col="split",
    aspect=1.5,
    height=8,
    sharex=False,
    sharey=False
)
# For each subplot make a line at y = 0 to show the error.
for ax in g.axes.flat:
    ax.axhline(0, ls='--', color='red')

# Adjust the layout
plt.subplots_adjust(top=0.85)
g.fig.suptitle('Absolute Soft/Hard Volumetric Log Error', fontsize=30)

# Show the plot
plt.show()

### Let's look at the same thing but this time also for Singapore.

In [ ]:
exp_2_df = melted_error_df.select(task='Singapore')

In [ ]:
g = sns.catplot(
    exp_2_df,
    x="data_id",
    y="error",
    hue="volume_type",
    col="split",
    aspect=1.5,
    height=8,
    sharex=False,
    sharey=False
)
# For each subplot make a line at y = 0 to show the error.
for ax in g.axes.flat:
    ax.axhline(0, ls='--', color='red')

# Adjust the layout
plt.subplots_adjust(top=0.85)
g.fig.suptitle('Singapore Soft/Hard Volumetric Error', fontsize=30)

# Show the plot
plt.show()

In [ ]:
g = sns.catplot(
    exp_2_df,
    x="data_id",
    y="log_abs_error",
    hue="volume_type",
    col="split",
    aspect=1.5,
    height=8,
    sharex=False,
    sharey=False,
)
# For each subplot make a line at y = 0 to show the error.
for ax in g.axes.flat:
    ax.axhline(0, ls='--', color='red')

# Adjust the layout
plt.subplots_adjust(top=0.85)
g.fig.suptitle('Singapore Absolute Soft/Hard Volumetric Log Error', fontsize=30)

# Show the plot
plt.show()